# Test of bound and algorithm

In [1]:
import numpy as np
import gym

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.algorithm.backward_feature_selection import BackwardFeatureSelector
from src.algorithm.info_theory.entropy import LeveOneOutEntropyEstimator, NNEntropyEstimator
from src import wenvs

In [4]:
est = NNEntropyEstimator()

In [159]:
env = gym.make('CartPole-v1')
env = wenvs.WrapperEnv(env, continuous_state=True)

wenv = wenvs.WrapperEnv(env, n_fake_features=1, n_fake_actions=1, continuous_state=True)
np.random.seed(0)
wenv.seed(0)

num_ep = 1000
trajectories = [np.hstack(wenv.run_episode()) for _ in range (num_ep)]

In [160]:
fs = BackwardFeatureSelector(est, [t for t in trajectories if len(t)>=10])
fs.selectOnError(10, 0.9, 1)

{0, 3, 4, 6}

In [153]:
fs.computeError()

1.5484439430837902